# ETTS Time-Series Forecasting with LMU

Train LMU model on Electricity Transformer Temperature (ETT) dataset.

In [ ]:
from __future__ import annotations
from typing import Tuple, Dict, Any
from torch.utils.data import DataLoader
from src.types.task_protocol import TaskProtocol
from src.datasets.etts.etts_dataloader import make_etts_loaders

class ETTSTask(TaskProtocol):
    problem_type: str = "regression"

    def make_loaders(self, data_root: str, batch_size: int = 64, num_workers: int = 4, **kwargs) -> Tuple[DataLoader, DataLoader, DataLoader]:
        return make_etts_loaders(
            data_root=data_root,
            which=kwargs.get("which", "ETTh1"),
            batch_size=batch_size,
            num_workers=num_workers,
            seq_len=kwargs.get("seq_len", 96),
            pred_len=kwargs.get("pred_len", 24),
            feature_mode=kwargs.get("feature_mode", "target"),
            target_col=kwargs.get("target_col", "OT"),
            split_ratio=kwargs.get("split_ratio", (0.7, 0.1, 0.2)),
            normalize=kwargs.get("normalize", "zscore"),
            pin_memory=kwargs.get("pin_memory", True),
            persistent_workers=kwargs.get("persistent_workers", False),
        )

    def infer_input_dim(self, args: Dict[str, Any]) -> int:
        fm = args.get("feature_mode", "target")
        return 1 if fm == "target_only" else 7

    def infer_num_classes(self, args: Dict[str, Any]) -> int:
        return 7 if args.get("feature_mode", "target") == "multivariate" else 1

    def infer_theta(self, args: Dict[str, Any]) -> int:
        return args.get("seq_len", 96)

def create_block_cfg_ctor(dropout, mlp_ratio, droppath_final, layerscale_init, residual_gain, pool):
    from src.models.v2.build_model import BlockConfig
    def block_cfg_ctor(theta: int):
        return BlockConfig(kind="lmu", memory_size=256, theta=theta, dropout=dropout, mlp_ratio=mlp_ratio,
                           droppath_final=droppath_final, layerscale_init=layerscale_init,
                           residual_gain=residual_gain, pool=pool)
    return block_cfg_ctor


## Configuration

In [ ]:
import torch
from pathlib import Path

current_dir = Path.cwd()
project_root = current_dir.parent.parent
data_root = str(project_root / "datasets" / "etts" / "data")

args = {
    "data_root": data_root,
    "batch": 64,
    "data_loader_kwargs": {
        "num_workers": 0,
        "which": "ETTh1",
        "seq_len": 96,
        "pred_len": 24,
        "feature_mode": "target",
        "target_col": "OT",
        "split_ratio": (0.7, 0.1, 0.2),
        "normalize": "zscore",
        "pin_memory": False,
        "persistent_workers": False,
    },
    "epochs": 50,
    "lr": 1e-3,
    "wd": 1e-4,
    "amp": True,
    "save_dir": "./runs/etts_task",
    "warmup_epochs": 5,
    "patience": 10,
    "min_delta": 0.001,
    "d_model": 256,
    "depth": 6,
    "dropout": 0.2,
    "mlp_ratio": 2.0,
    "droppath_final": 0.1,
    "layerscale_init": 1e-2,
    "residual_gain": 1.0,
    "pool": "none",
}

args["block_cfg_ctor"] = create_block_cfg_ctor(
    dropout=args["dropout"], mlp_ratio=args["mlp_ratio"], droppath_final=args["droppath_final"],
    layerscale_init=args["layerscale_init"], residual_gain=args["residual_gain"], pool=args["pool"]
)

if torch.backends.mps.is_available():
    args["device"] = torch.device("mps")
    print("🚀 Using MPS (M4 Neural Engine)")
elif torch.cuda.is_available():
    args["device"] = torch.device("cuda")
else:
    args["device"] = torch.device("cpu")
    args["amp"] = False

## Training